This will be used for the ibm data science course capstone


In [1]:
import numpy as np
import pandas as pd
#IMPORT BeautifulSoup library
from bs4 import BeautifulSoup
import requests
import wikipedia as wp
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium




In [2]:
print("hello capstone project course!")

hello capstone project course!


In [3]:
tor_df = pd.read_csv('Toronto PC.csv')
tor_df.head()

,Postal Code,Place Name,Province,Latitude,Longitude,Unnamed: 5,Unnamed: 6
0,M3K,Downsview East (CFB Toronto),ON,43.7390,-79.4692,NaN,NaN
1,M4E,East Toronto (The Beaches),ON,43.6784,-79.2941,NaN,NaN
2,M4J,East Toronto (The Danforth East),ON,43.6872,-79.3368,NaN,NaN
3,M4K,East Toronto (The Danforth West / Riverdale),ON,43.6803,-79.3538,NaN,NaN
4,M4L,East Toronto (India Bazaar / The Beaches West),ON,43.6693,-79.3155,NaN,NaN


In [4]:
tor_df.rename(columns={'Place Name': 'Neighborhood'}, inplace=True)

In [5]:
tor_df.drop(columns=['Unnamed: 5','Unnamed: 6'],axis=1,inplace=True)
tor_df.head()

,Postal Code,Neighborhood,Province,Latitude,Longitude
0,M3K,Downsview East (CFB Toronto),ON,43.7390,-79.4692
1,M4E,East Toronto (The Beaches),ON,43.6784,-79.2941
2,M4J,East Toronto (The Danforth East),ON,43.6872,-79.3368
3,M4K,East Toronto (The Danforth West / Riverdale),ON,43.6803,-79.3538
4,M4L,East Toronto (India Bazaar / The Beaches West),ON,43.6693,-79.3155


In [6]:
neighborhood_latitude = tor_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = tor_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = tor_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Downsview East (CFB Toronto) are 43.739, -79.4692.


In [7]:
CLIENT_ID = ('J3NOHS1T0JG3MPSG4WFZ5IQT4ICUNFZF3DVH4UTI425PL4HR')
CLIENT_SECRET = ('YL4PGSYANHHFQI53S1YKRWVMUTUQUIFTMRX0LPTYJAVJGGXH')
VERSION='20180323'


In [8]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=J3NOHS1T0JG3MPSG4WFZ5IQT4ICUNFZF3DVH4UTI425PL4HR&client_secret=YL4PGSYANHHFQI53S1YKRWVMUTUQUIFTMRX0LPTYJAVJGGXH&v=20180323&ll=43.739,-79.4692&radius=500&limit=100'

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60b8df8687bfd807c8e5af88'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Clanton Park',
  'headerFullLocation': 'Clanton Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.7435000045, 'lng': -79.4629832208559},
   'sw': {'lat': 43.734499995499995, 'lng': -79.4754167791441}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bcb76143740b7133e926265',
       'name': 'Toronto Downsview Airport (YZD)',
       'contact': {},
       'location': {'address': 'Garratt Blvd',
        'lat': 43.738882611749744,
        'lng': -79.47011109314882,
        'labeledLatLngs': [{'label': '

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-11-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Toronto Downsview Airport (YZD),Airport,43.738883,-79.470111
1,Tim Hortons,Coffee Shop,43.738890,-79.472242
2,Main Cafeteria (Bombardier),Food Court,43.737931,-79.471144
3,Work Authority Truck,Shoe Store,43.738352,-79.471896


In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=tor_df['Neighborhood'],
                                   latitudes=tor_df['Latitude'],
                                   longitudes=tor_df['Longitude']
                                  )

Downsview East (CFB Toronto)
East Toronto (The Beaches)
East Toronto (The Danforth East)
East Toronto (The Danforth West / Riverdale)
East Toronto (India Bazaar / The Beaches West)
East Toronto (Studio District)
Central Toronto (Lawrence Park East)
Central Toronto (Davisville North)
Central Toronto (North Toronto West)
Central Toronto (Davisville)
Central Toronto (Moore Park / Summerhill East)
Central Toronto (Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park)
Downtown Toronto (Rosedale)
Downtown Toronto (St. James Town / Cabbagetown)
Downtown Toronto (Church and Wellesley)
Downtown Toronto (Regent Park / Port of Toronto)
Downtown Toronto (Ryerson)
Downtown Toronto (St. James Park)
Downtown Toronto (Berczy Park)
Downtown Toronto (Central Bay Street)
Downtown Toronto (Richmond / Adelaide / King)
Downtown Toronto (Harbourfront East / Union Station / Toronto Island)
Downtown Toronto (Toronto Dominion Centre / Design Exchange)
Downtown Toronto (Commerce Court / Victoria

In [16]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 214 unique categories.


In [ ]:
toronto_venues.head()

In [17]:
# one hot encoding
tor_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [18]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,Central Toronto (Davisville North),0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Central Toronto (Davisville),0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Toronto (Forest Hill North & West),0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Toronto (Lawrence Park East),0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Toronto (Moore Park / Summerhill East),0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Central Toronto (North Toronto West),0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Toronto (Roselawn),0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Toronto (Summerhill West / Rathnelly /...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Central Toronto (The Annex / North Midtown / Y...,0.000000,0.000000,0.000000,0.000000,0.00,0.047619,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Downsview East (CFB Toronto),0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto (Davisville North)----
               venue  freq
0     Breakfast Spot  0.14
1  Food & Drink Shop  0.14
2         Playground  0.14
3   Department Store  0.14
4               Park  0.14


----Central Toronto (Davisville)----
                venue  freq
0      Sandwich Place  0.09
1        Dessert Shop  0.09
2         Coffee Shop  0.09
3                Café  0.09
4  Italian Restaurant  0.09


----Central Toronto (Forest Hill North & West)----
               venue  freq
0              Trail   0.5
1  Electronics Store   0.5
2        Yoga Studio   0.0
3        Music Store   0.0
4      Metro Station   0.0


----Central Toronto (Lawrence Park East)----
                venue  freq
0  Photography Studio   0.5
1                Park   0.5
2         Music Store   0.0
3       Metro Station   0.0
4  Mexican Restaurant   0.0


----Central Toronto (Moore Park / Summerhill East)----
             venue  freq
0             Park   0.4
1              Gym   0.2
2    Grocery Store   0.2
3

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto (Davisville North),Gym / Fitness Center,Dog Run,Park,Playground,Breakfast Spot,Food & Drink Shop,Department Store,Wings Joint,Escape Room,Eastern European Restaurant
1,Central Toronto (Davisville),Coffee Shop,Sandwich Place,Café,Italian Restaurant,Dessert Shop,Fast Food Restaurant,Farmers Market,Restaurant,Sushi Restaurant,Seafood Restaurant
2,Central Toronto (Forest Hill North & West),Trail,Electronics Store,Wings Joint,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
3,Central Toronto (Lawrence Park East),Photography Studio,Park,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
4,Central Toronto (Moore Park / Summerhill East),Park,Thai Restaurant,Gym,Grocery Store,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


# Clustering

In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 4, 2, 2, 3, 0, 0, 0], dtype=int32)

In [55]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ValueError: cannot insert Cluster Labels, already exists

In [41]:
tor_merged[:-1]

,Postal Code,Neighborhood,Province,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3K,Downsview East (CFB Toronto),ON,44,-79,0,Food Court,Airport,Shoe Store,Coffee Shop,Wings Joint,Doner Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4E,East Toronto (The Beaches),ON,44,-79,0,Pub,Health Food Store,Trail,Gastropub,Bakery,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
2,M4J,East Toronto (The Danforth East),ON,44,-79,0,Convenience Store,Greek Restaurant,Park,Intersection,Wings Joint,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
3,M4K,East Toronto (The Danforth West / Riverdale),ON,44,-79,0,Greek Restaurant,Ice Cream Shop,Café,Italian Restaurant,Coffee Shop,Grocery Store,Bubble Tea Shop,Restaurant,Pub,Cocktail Bar
4,M4L,East Toronto (India Bazaar / The Beaches West),ON,44,-79,0,Park,Sandwich Place,Restaurant,Fast Food Restaurant,Liquor Store,Steakhouse,Sushi Restaurant,Food & Drink Shop,Pub,Ice Cream Shop
5,M4M,East Toronto (Studio District),ON,44,-79,0,Coworking Space,Garden Center,Gym,Park,Coffee Shop,Baseball Field,Diner,Eastern European Restaurant,Donut Shop,Dumpling Restaurant
6,M4N,Central Toronto (Lawrence Park East),ON,44,-79,4,Photography Studio,Park,Dog Run,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
7,M4P,Central Toronto (Davisville North),ON,44,-79,0,Gym / Fitness Center,Dog Run,Park,Playground,Breakfast Spot,Food & Drink Shop,Department Store,Wings Joint,Escape Room,Eastern European Restaurant
8,M4R,Central Toronto (North Toronto West),ON,44,-79,2,Gym Pool,Garden,Playground,Park,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
9,M4S,Central Toronto (Davisville),ON,44,-79,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Dessert Shop,Fast Food Restaurant,Farmers Market,Restaurant,Sushi Restaurant,Seafood Restaurant


In [56]:
tor_merged.dtypes

Postal Code                object
Neighborhood               object
Province                   object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [57]:
# create map
map_clusters = folium.Map(location=[43.651070,- 79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters